Курсовая работа - основы языка программирования Python

In [11]:
import requests
import time
import pandas as pd

class VK:

    def __init__(self, access_token, user_id, version='5.131'):
        self.token = access_token
        self.id = user_id
        self.version = version
        self.params = {'access_token': self.token, 'v': self.version}
        self.base_url = 'https://api.vk.com/method/'

    def users_info(self):
        """
        Метод получает базовую информацию о пользователе
        """
        url = self.base_url + 'users.get'
        response = requests.get(url, params=self.params)
        return response
    
    def photos_get():
        url = self.base_url + 'photos.get'
        photos_params = {
            'owner_id': -1,
            'album_id': 'wall',
            'count'   : 2
        }
        response = requests.get(url, params={**self.params, **photos_params})
        return response
        

access_token = '...'
user_id = '...'
vk = VK(access_token, user_id)
# Получить базовую информацию о пользователе
basic_user_information = pd.DataFrame(vk.users_info().json()['response'])
basic_user_information
vk.photos_get().json()

TypeError: photos_get() takes 0 positional arguments but 1 was given